In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

src_words_size = 11  # 源语言词表大小
tgt_words_size = 10  # 目标语言词表大小

torch.manual_seed(42)

# 序列建模的序列大小
max_src_len = 10
max_tgt_len = 10
max_position_len = max(max_src_len, max_tgt_len)

# 词嵌入的维度， transformer原始是512维
model_dim = 8

# 假设现在有这么一个翻译任务， 比如将我爱你翻译为i love you , 下边的第一个序列表示输入序列, 第二个序列表示输出序列
# 对应的长度就是3 - 3 (假设我 爱 你 分别是三个token)
# 那么输入序列和输出序列可以表示为如下的索引序列
input_seq_len = torch.Tensor([3, 2, 5]).to(torch.int32) # 输入序列的实际长度
target_seq_len = torch.Tensor([3, 2, 6]).to(torch.int32) # 输出序列的实际长度

# 构建输入序列, 表示在词表中的索引
# torch.randint(low=0, high, size, dtype=None, device=None, requires_grad=False)
input_seq = [torch.randint(1, src_words_size, (l,)) for l in input_seq_len]
# 构建输出序列, 表示在词表中的索引
output_seq = [torch.randint(1, tgt_words_size, (l,)) for l in target_seq_len]

print(f'input_seq = {input_seq} \nout_seq = {output_seq}')

# 为了保证输入序列和输出序列在一个batch中具有相同的长度, 需要对序列进行padding操作
for idx, t in enumerate(input_seq):
    input_seq[idx] = F.pad(t, (0, max_src_len - t.size(0)),value = 0)

for idx, t in enumerate(output_seq):
    output_seq[idx] = F.pad(t, (0, max_tgt_len - t.size(0)),value = 0)

# print(f'Padded input_seq = {input_seq} \nPadded out_seq = {output_seq}')

# 接着我们要将上面这些list转化为一个整体的输入batch, 即一个tensor

input_seq = torch.stack(input_seq, dim=0)  # shape: (batch_size, max_src_len)
output_seq = torch.stack(output_seq, dim=0)  # shape: (batch_size, max_tgt_len)

input_seq = [tensor([3, 8, 7]), tensor([5, 7]), tensor([6, 1, 5, 1, 4])] 
out_seq = [tensor([5, 5, 9]), tensor([1, 1]), tensor([5, 3, 5, 4, 5, 5])]


In [2]:
'''
完成拼接之后的两个tensor
'''

print(f'Final input_seq: \n{input_seq} \nFinal out_seq: \n{output_seq}')

Final input_seq: 
tensor([[3, 8, 7, 0, 0, 0, 0, 0, 0, 0],
        [5, 7, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 1, 5, 1, 4, 0, 0, 0, 0, 0]]) 
Final out_seq: 
tensor([[5, 5, 9, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [5, 3, 5, 4, 5, 5, 0, 0, 0, 0]])


In [3]:
# 构建词嵌入表
input_embedding_table = nn.Embedding(src_words_size, model_dim, padding_idx=0)
output_embedding_table = nn.Embedding(tgt_words_size, model_dim, padding_idx=0)

# 展示embedding的效果
# print(input_embedding_table.weight)
# print(output_embedding_table.weight)

input_embedded = input_embedding_table(input_seq)  # shape: (batch_size, max_src_len, model_dim)
output_embedded = output_embedding_table(output_seq)  # shape: (batch_size, max_tgt_len, model_dim)

print(input_embedded[0]) # 展示第一个样本的词嵌入结果


tensor([[ 2.1228, -1.2347, -0.4879, -1.4181,  0.8963,  0.0499,  2.2667,  1.1790],
        [ 0.5246,  1.1412,  0.0516,  0.7281, -0.7106, -0.6021,  0.9604,  0.4048],
        [-0.6417, -2.2064, -0.7508,  2.8140,  0.3598, -0.0898,  0.4584, -0.5644],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<SelectBackward0>)


### BatchNorm 和 LayerNorm有什么区别

Batch Normalization (BN) 和 Layer Normalization (LN) 都是深度学习中用于加速训练、防止过拟合和解决梯度消失/爆炸的归一化技术。

它们的核心区别在于归一化的维度不同。简单来说：

BatchNorm 是“竖”着切（跨样本，同特征）

LayerNorm 是“横”着切（同样本，跨特征）

1. 核心原理区别

    假设我们有一批数据，形状为 [Batch_Size, Features]（比如 32 个样本，每个样本 100 个特征）。Batch Normalization (BN)计算方式：它在*Batch维度*上进行归一化。具体操作：对于第 $i$ 个特征，它会计算该 Batch 中所有 32 个样本在该特征上的均值和方差，然后把这个特征归一化。直观理解：BN 强行让这一批数据在每一个特征维度上都变成标准正态分布。它依赖于其他样本来确定当前样本的相对位置。Layer Normalization (LN)计算方式：它在 Feature (Layer) 维度 上进行归一化。具体操作：对于第 $j$ 个样本，它会计算该样本自身所有 100 个特征的均值和方差，然后把自己归一化。直观理解：LN 是“独立自主”的。它不看别的样本，只看自己内部特征的分布情况。

In [4]:
# 位置编码
pos_vector = torch.arange(0, max_position_len).reshape((-1,1))
print(f'pos_vector = {pos_vector}')
dim_indices = torch.arange(0, model_dim, 2).float()
# pos_vector.shape = (10, 1)
# [[0.], [1.], [2.], ..., [9.]]

i_matrix = torch.pow(10000, (dim_indices*2 / model_dim))
print(f'i_matrix transformed = {i_matrix}')

# 获取位置编码的表
position_encoding_table = torch.zeros(max_position_len, model_dim)
position_encoding_table[:, 0::2] = torch.sin(pos_vector / i_matrix)
position_encoding_table[:, 1::2] = torch.cos(pos_vector / i_matrix)

print(f'position_encoding_table = {position_encoding_table}')

# 实际的位置编码, 形状是(max_position_len, model_dim)
# 
pe_embeding = nn.Embedding(max_position_len, model_dim)
pe_embeding.weight = nn.Parameter(position_encoding_table, requires_grad=False)

pos_vector = tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])
i_matrix transformed = tensor([1.0000e+00, 1.0000e+02, 1.0000e+04, 1.0000e+06])
position_encoding_table = tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9998e-03,  9.9995e-01,  1.0000e-04,
          1.0000e+00,  1.0000e-06,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9999e-02,  9.9980e-01,  2.0000e-04,
          1.0000e+00,  2.0000e-06,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9995e-02,  9.9955e-01,  3.0000e-04,
          1.0000e+00,  3.0000e-06,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.9989e-02,  9.9920e-01,  4.0000e-04,
          1.0000e+00,  4.0000e-06,  1.0000e+00],
        [-9.5892e-01,  2.8366e-01,  4.9979e-02,  9.9875e-01,  5.0000e-04,
          1.0000e+00,  5.0000e-06,  1.0000e+00],
        [-2.

现在位置编码做的事情其实是获取单词 比如第一行的'3'在当前句子中的位置， 也就是 **在句子中的索引**

In [5]:
print(f'input_seq = \n{input_seq}')

# 1. 获取维度信息
batch_size = input_seq.size(0)  # 3
seq_len = input_seq.size(1)     # 10

# 2. 生成位置索引 [0, 1, 2, ..., 9]
# device参数是为了保证如果 input_seq 在GPU上，位置索引也在GPU上
pos = torch.arange(seq_len, dtype=torch.long)

# 3. 扩展维度以匹配 (batch_size, seq_len)
# 先变成 (1, 10)，然后复制 3 次变成 (3, 10)
input_pos = pos.unsqueeze(0).repeat(batch_size, 1)

print(f'input_pos = \n{input_pos}')
# 预期输出:
# tensor([[0, 1, 2, ..., 9],
#         [0, 1, 2, ..., 9],
#         [0, 1, 2, ..., 9]])

# 4. 送入 Embedding 层
input_pe_embedded = pe_embeding(input_pos)

print(f'input_pe_embedded shape = {input_pe_embedded}')
# 预期输出: torch.Size([3, 10, 8])

input_seq = 
tensor([[3, 8, 7, 0, 0, 0, 0, 0, 0, 0],
        [5, 7, 0, 0, 0, 0, 0, 0, 0, 0],
        [6, 1, 5, 1, 4, 0, 0, 0, 0, 0]])
input_pos = 
tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
input_pe_embedded shape = tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9998e-03,  9.9995e-01,  1.0000e-04,
           1.0000e+00,  1.0000e-06,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9999e-02,  9.9980e-01,  2.0000e-04,
           1.0000e+00,  2.0000e-06,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9995e-02,  9.9955e-01,  3.0000e-04,
           1.0000e+00,  3.0000e-06,  1.0000e+00],
         [-7.5680e-01, -6.5364e-01,  3.9989e-02,  9.9920e-01,  4.0000e-04,
           1.0000e+00,  4.0000e-06,  1.0000e+00],
         [-9.5892e-01,  2.8366e-01,  4.9979e-02,  9.9875e-01,  5.0000e-04,
      

In [6]:
# 对于 output_seq (目标序列)，逻辑完全一样
tgt_len = output_seq.size(1) # 10

# 1. 生成位置索引 [0, 1, ..., 9]
tgt_pos = torch.arange(tgt_len, dtype=torch.long, device=output_seq.device)

# 2. 扩展维度 (3, 10)
output_pos = tgt_pos.unsqueeze(0).repeat(batch_size, 1)

print(f'output_pos = \n{output_pos}')

# 3. 送入 Embedding 层
output_pe_embedded = pe_embeding(output_pos)

print(f'output_pe_embedded shape = {output_pe_embedded.shape}')
# 预期输出: torch.Size([3, 10, 8])

output_pos = 
tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
output_pe_embedded shape = torch.Size([3, 10, 8])


In [7]:
# 构造encoder的self-attention mask， 因为我们的位置编码时，padding填充了0，所以需要表示一下每个句子的有效位置

valid_pos = [torch.unsqueeze(F.pad(torch.ones(l), (0, max_position_len - l)), 0) for l in input_seq_len]
valid_pos = torch.cat(valid_pos, dim=0).unsqueeze(2)  # shape: (batch_size, max_position_len)

mask_attention_matrix = torch.bmm(valid_pos, valid_pos.transpose(1,2))  # shape: (batch_size, max_position_len, max_position_len)

In [9]:
# 接着我们假设计算出来的注意力分数 是一个随机的矩阵

attention_score = torch.randn(batch_size, max_position_len, max_position_len)

# 应用mask到注意力分数上
masked_attention_score = attention_score.masked_fill(mask_attention_matrix == 0,1e-9)

# 计算softmax分数
attention_weights = F.softmax(masked_attention_score, dim=-1)

attention_weights.shape

torch.Size([3, 10, 10])

In [ ]:
# 同样对于out_put我们也需要构造一下mask矩阵,参考上面的步骤
output_valid_pos = [torch.unsqueeze(F.pad(torch.ones(l), (0, max_position_len - l)), 0) for l in target_seq_len]
output_valid_pos = torch.cat(output_valid_pos, dim=0).unsqueeze(2)  # shape: (batch_size, max_position_len, 1)

output_valid_pos.shape

output_valid_mask_matrix = torch.bmm(output_valid_pos, output_valid_pos.transpose(1,2))  # shape: (batch_size, max_position_len, max_position_len)

# 三个矩阵分别表示每个句子第x个字与第n个之间是否有效， 即是否需要计算这两个字的attention score
output_valid_mask_matrix


tensor([[[1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.,

In [30]:
# 我们还需要构造一个模拟自回归的mask矩阵

aggressive_mask_list = [F.pad(torch.tril(torch.ones((L, L))), (0, max(target_seq_len - L),0 , max(target_seq_len - L)))\
     for L in target_seq_len]

# 由于每个词的形状不一致， 所以我们还需要进行padding操作，保证每一个的mask大小是相同的
# print(aggressive_mask_list)
aggressive_mask_matrix = torch.stack(aggressive_mask_list, dim=0)

print(f'naggressive_mask_matrix shape: {aggressive_mask_matrix.shape}\naggressive_mask_matrix = \n{aggressive_mask_matrix}')

naggressive_mask_matrix shape: torch.Size([3, 6, 6])
aggressive_mask_matrix = 
tensor([[[1., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1.]]])


行 (Query 轴),代表当前正在计算/预测的词（Q 向量的来源）。
列 (Key/Value 轴),代表可以被当前词关注/查看的词（K/V 向量的来源）。